In [1]:
# Importamos las librerías que vamos a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import sklearn_pandas
from matplotlib import gridspec

In [2]:
# Quitamos el límite de columnas y filas que se muestran en los dataframes
pd.set_option('display.max_columns', None)

In [3]:
# Importamos los csvs que vamos a utilizar
races = pd.read_csv('./csvs/races.csv')
circuits = pd.read_csv('./csvs/circuits.csv')
results = pd.read_csv('./csvs/results.csv')
driver_standings = pd.read_csv('./csvs/driver_standings.csv')
constructor_standings = pd.read_csv('./csvs/constructor_standings.csv')
qualifying = pd.read_csv('./csvs/qualifying.csv')

In [4]:
# Eliminamos las columnas que no vamos a utilizar en cada dataset
races.drop(['year', 'round', 'name', 'url', 'fp1_date', 'fp1_time','fp2_date', 'fp2_time','fp3_date',
            'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'], axis=1, inplace=True)

circuits.drop(['name', 'location', 'country', 'lat', 'lng', 'url'], axis=1, inplace=True)

results.drop(['number', 'position', 'positionText', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 
           'fastestLapTime', 'fastestLapSpeed', 'statusId', 'rank'], axis=1, inplace=True)

driver_standings.drop(['positionText'], axis=1, inplace=True)

constructor_standings.drop(['positionText'], axis=1, inplace=True)

qualifying.drop(['number'], axis=1, inplace=True)

In [5]:
results.head()

,resultId,raceId,driverId,constructorId,grid,positionOrder
0,1,18,1,1,1,1
1,2,18,2,2,5,2
2,3,18,3,3,7,3
3,4,18,4,4,11,4
4,5,18,5,1,3,5


In [6]:
# Seteamos como índice de los dataframes la columna por la que vamos a realizar joins
races.set_index('raceId')
circuits.set_index('circuitId')
results.set_index('raceId')
driver_standings.set_index('raceId')
constructor_standings.set_index('raceId')
qualifying.set_index('raceId')

,qualifyId,driverId,constructorId,position,q1,q2,q3
raceId,,,,,,,
18,1,1,1,1,1:26.572,1:25.187,1:26.714
18,2,9,2,2,1:26.103,1:25.315,1:26.869
18,3,5,1,3,1:25.664,1:25.452,1:27.079
18,4,13,6,4,1:25.994,1:25.691,1:27.178
18,5,2,2,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...
1096,9628,825,210,16,1:25.834,\N,\N
1096,9629,842,213,17,1:25.859,\N,\N
1096,9630,822,51,18,1:25.892,\N,\N


In [7]:
# Cambiamos nombres de columnas para identificarlos mejor
qualifying.rename(columns = {'position':'q_position'}, inplace = True)
driver_standings.rename(columns = {'points':'ds_points', 'position':'ds_position', 'wins':'ds_wins'}, inplace = True)
constructor_standings.rename(columns = {'points':'cs_points', 'position':'cs_position', 'wins':'cs_wins'}, inplace = True)
circuits.rename(columns = {'alt':'altitud'}, inplace = True)

In [24]:
full = pd.merge(results, qualifying, how='inner', on=['raceId', 'driverId', 'constructorId']).merge(
    driver_standings, how='inner', on=['raceId', 'driverId']).merge(
    constructor_standings, how='inner', on=['raceId', 'constructorId']).merge(races, how='inner', on=['raceId']).merge(
    circuits, how='inner', on=['circuitId'])

In [29]:
full.head()

,resultId,raceId,driverId,constructorId,grid,positionOrder,qualifyId,q_position,q1,q2,q3,driverStandingsId,ds_points,ds_position,ds_wins,constructorStandingsId,cs_points,cs_position,cs_wins,circuitId,date,time,circuitRef,altitud
0,1,18,1,1,1,1,1,1,1:26.572,1:25.187,1:26.714,1,10.0,1,1,1,14.0,1,1,1,2008-03-16,04:30:00,albert_park,10
1,5,18,5,1,3,5,3,3,1:25.664,1:25.452,1:27.079,5,4.0,5,0,1,14.0,1,1,1,2008-03-16,04:30:00,albert_park,10
2,2,18,2,2,5,2,5,5,1:25.960,1:25.518,1:27.236,2,8.0,2,0,2,8.0,3,0,1,2008-03-16,04:30:00,albert_park,10
3,3,18,3,3,7,3,7,7,1:26.295,1:26.059,1:28.687,3,6.0,3,0,3,9.0,2,0,1,2008-03-16,04:30:00,albert_park,10
4,6,18,6,3,13,6,14,14,1:26.891,1:26.413,\N,6,3.0,6,0,3,9.0,2,0,1,2008-03-16,04:30:00,albert_park,10


In [28]:
full.shape

(9322, 24)

In [30]:
full.dtypes

resultId                    int64
raceId                      int64
driverId                    int64
constructorId               int64
grid                        int64
positionOrder               int64
qualifyId                   int64
q_position                  int64
q1                         object
q2                         object
q3                         object
driverStandingsId           int64
ds_points                 float64
ds_position                 int64
ds_wins                     int64
constructorStandingsId      int64
cs_points                 float64
cs_position                 int64
cs_wins                     int64
circuitId                   int64
date                       object
time                       object
circuitRef                 object
altitud                    object
dtype: object

In [37]:
full['positionOrder'].isnull().values.any()

False

In [44]:
# Creamos la columna 'is_podium' que es el target a predecir a partir de los datos que tenemos en positionOrder
full['is_podium'] = np.where(full['positionOrder'] <= 3, 1, 0)
full.drop(['positionOrder'], axis=1, inplace=True)
full.head()

,resultId,raceId,driverId,constructorId,grid,qualifyId,q_position,q1,q2,q3,driverStandingsId,ds_points,ds_position,ds_wins,constructorStandingsId,cs_points,cs_position,cs_wins,circuitId,date,time,circuitRef,altitud,is_podium
0,1,18,1,1,1,1,1,1:26.572,1:25.187,1:26.714,1,10.0,1,1,1,14.0,1,1,1,2008-03-16,04:30:00,albert_park,10,1
1,5,18,5,1,3,3,3,1:25.664,1:25.452,1:27.079,5,4.0,5,0,1,14.0,1,1,1,2008-03-16,04:30:00,albert_park,10,0
2,2,18,2,2,5,5,5,1:25.960,1:25.518,1:27.236,2,8.0,2,0,2,8.0,3,0,1,2008-03-16,04:30:00,albert_park,10,1
3,3,18,3,3,7,7,7,1:26.295,1:26.059,1:28.687,3,6.0,3,0,3,9.0,2,0,1,2008-03-16,04:30:00,albert_park,10,1
4,6,18,6,3,13,14,14,1:26.891,1:26.413,\N,6,3.0,6,0,3,9.0,2,0,1,2008-03-16,04:30:00,albert_park,10,0


In [45]:
full.shape

(9322, 24)

In [46]:
full.dtypes

resultId                    int64
raceId                      int64
driverId                    int64
constructorId               int64
grid                        int64
qualifyId                   int64
q_position                  int64
q1                         object
q2                         object
q3                         object
driverStandingsId           int64
ds_points                 float64
ds_position                 int64
ds_wins                     int64
constructorStandingsId      int64
cs_points                 float64
cs_position                 int64
cs_wins                     int64
circuitId                   int64
date                       object
time                       object
circuitRef                 object
altitud                    object
is_podium                   int32
dtype: object